<a href="https://colab.research.google.com/github/Sakib56/TTDS-G35-CW3/blob/main/draft_vector_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Uncomment and run the following cells if you work on Google Colab :) Don't forget to change your runtime type to GPU!

In [1]:
!git clone https://github.com/kstathou/vector_engine

Cloning into 'vector_engine'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 74 (delta 32), reused 59 (delta 18), pack-reused 0
Unpacking objects: 100% (74/74), done.


In [2]:
cd vector_engine

/content/vector_engine


In [3]:
pip install -r requirements.txt

Obtaining file:///content/vector_engine (from -r requirements.txt (line 9))
     |████████████████████████████████| 804.1 MB 2.8 kB/s 
     |████████████████████████████████| 1.1 MB 36.8 MB/s 
     |████████████████████████████████| 66 kB 6.7 MB/s 
     |████████████████████████████████| 10.5 MB 54.7 MB/s 
     |████████████████████████████████| 7.1 MB 73.9 MB/s 
     |████████████████████████████████| 14.5 MB 13.1 MB/s 
     |████████████████████████████████| 69 kB 9.4 MB/s 
     |████████████████████████████████| 7.1 MB 10.3 MB/s 
     |████████████████████████████████| 895 kB 51.7 MB/s 
     |████████████████████████████████| 1.2 MB 60.3 MB/s 
     |████████████████████████████████| 3.0 MB 44.9 MB/s 
     |████████████████████████████████| 8.6 MB 58.9 MB/s 
     |████████████████████████████████| 4.3 MB 56.7 MB/s 
     |████████████████████████████████| 132 kB 70.4 MB/s 
     |████████████████████████████████| 111 kB 82.3 MB/s 
     |████████████████████████████████| 76 kB 6.3 MB/s 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Let's begin!

In [2]:
%load_ext autoreload

In [18]:
pip install --upgrade pandas

     |████████████████████████████████| 11.3 MB 14.8 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.2
    Uninstalling pandas-1.1.2:
      Successfully uninstalled pandas-1.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.9.1 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.32.0 which is incompatible.


In [3]:
%autoreload 2
# Used to import data from local.
import pandas as pd

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle
from pathlib import Path

# Used to do vector searches and display the results.
from vector_engine.utils import vector_search, id2details

In [39]:
df = pd.read_xml('/content/drive/MyDrive/Colab Notebooks/wikidata_short.xml', xpath=".//revision")
df2 = pd.read_xml('/content/drive/MyDrive/Colab Notebooks/wikidata_short.xml')
df = df.drop("id",1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [40]:
final_df = pd.concat([df2, df], axis=1)

In [41]:
final_df = final_df.dropna(axis=1)

In [54]:
final_df.head(3)

,title,ns,id,timestamp,model,format,text,sha1
0,Anarchism,0,12,2021-10-29T21:58:07Z,wikitext,text/x-wiki,{{short description|Political philosophy and m...,6jpem2q0igox3vs9lrul8v3ugsn98cs
1,AfghanistanHistory,0,13,2017-06-05T04:18:18Z,wikitext,text/x-wiki,#REDIRECT [[History of Afghanistan]]\n\n{{Redi...,iv7s0lr40b17x33tfb0caa2g5bplgrk
2,AfghanistanGeography,0,14,2017-06-05T04:18:23Z,wikitext,text/x-wiki,#REDIRECT [[Geography of Afghanistan]]\n\n{{Re...,39r4w8qg62iexlyskf0ga3tblagtl8x


In [44]:
print(f"Wikidata papers: {final_df.id.unique().shape[0]}")

Wikidata papers: 339


The [Sentence Transformers library](https://github.com/UKPLab/sentence-transformers) offers pretrained transformers that produce SOTA sentence embeddings. Checkout this [spreadsheet](https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/) with all the available models.

In this tutorial, we will use the `distilbert-base-nli-stsb-mean-tokens` model which has the best performance on Semantic Textual Similarity tasks among the DistilBERT versions. Moreover, although it's slightly worse than BERT, it is quite faster thanks to having a smaller size.

I use the same model in [Orion's semantic search engine](https://www.orion-search.org/)!

In [45]:
# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

100%|██████████| 245M/245M [00:22<00:00, 10.7MB/s]


cuda:0


In [47]:
# Convert abstracts to vectors
embeddings = model.encode(final_df.text.to_list(), show_progress_bar=True)

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

In [48]:
print(f'Shape of the vectorised abstract: {embeddings[0].shape}')

Shape of the vectorised abstract: (768,)


## Vector similarity search with Faiss
[Faiss](https://github.com/facebookresearch/faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, even ones that do not fit in RAM. 
    
Faiss is built around the `Index` object which contains, and sometimes preprocesses, the searchable vectors. Faiss has a large collection of [indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes). You can even create [composite indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes-(composite)). Faiss handles collections of vectors of a fixed dimensionality d, typically a few 10s to 100s.

**Note**: Faiss uses only 32-bit floating point matrices. This means that you will have to change the data type of the input before building the index.

To learn more about Faiss, you can read their paper on [arXiv](https://arxiv.org/abs/1702.08734).

Here, we will the `IndexFlatL2` index:
- It's a simple index that performs a brute-force L2 distance search
- It scales linearly. It will work fine with our data but you might want to try [faster indexes](https://github.com/facebookresearch/faiss/wiki/Faster-search) if you work will millions of vectors.

To create an index with the `misinformation` abstract vectors, we will:
1. Change the data type of the abstract vectors to float32.
2. Build an index and pass it the dimension of the vectors it will operate on.
3. Pass the index to IndexIDMap, an object that enables us to provide a custom list of IDs for the indexed vectors.
4. Add the abstract vectors and their ID mapping to the index. In our case, we will map vectors to their paper IDs from MAG.

In [50]:
# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, final_df.id.values)

print(f"Number of vectors in the Faiss index: {index.ntotal}")

Number of vectors in the Faiss index: 339


### Searching the index
The index we built will perform a k-nearest-neighbour search. We have to provide the number of neighbours to be returned. 

Let's query the index with an abstract from our dataset and retrieve the 10 most relevant documents. **The first one must be our query!**


In [56]:
# Paper abstract
final_df.iloc[300, 6]

'#REDIRECT [[Lycoming T55]]\n\n{{Redirect category shell|\n{{R from incorrect name}}\n{{R from move}}\n}}'

In [57]:
# Retrieve the 10 nearest neighbours
D, I = index.search(np.array([embeddings[300]]), k=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

L2 distance: [0.0, 91.28944396972656, 92.35748291015625, 93.8724136352539, 95.19764709472656, 95.42794036865234, 96.56440734863281, 96.89290618896484, 97.49163055419922, 97.81593322753906]

MAG paper IDs: [69171661, 69171578, 69171555, 69171739, 42, 69171442, 69171463, 272, 53, 247]


In [59]:
# Fetch the paper titles based on their index
id2details(final_df, I, 'title')

[['Honeywell T55 Turboshaft Engine'],
 ['Draft:Vasudha Dhagamwar'],
 ['Masdar Institute of Science and Technology'],
 ['Quan khi Dao'],
 ['ArtificalLanguages'],
 ["Principauté d'Aigues-Mortes"],
 ['Henri Louit'],
 ['AcademicElitism'],
 ['Abbreviations'],
 ['AbeceDarians']]

In [61]:
# Fetch the paper abstracts based on their index
id2details(final_df, I, 'text')

[['#REDIRECT [[Lycoming T55]]\n\n{{Redirect category shell|\n{{R from incorrect name}}\n{{R from move}}\n}}'],
 ['#REDIRECT [[Vasudha Dhagamwar]]\n\n{{Redirect category shell|\n{{R from move}}\n}}'],
 ['#REDIRECT [[Masdar Institute]]\n\n{{Redirect category shell|\n{{R from move}}\n{{R from shorter name}}\n{{R from alternative name}}\n}}'],
 ['#REDIRECT [[Qwan Ki Do]]\n{{Rcat shell|{{R from alternative name}}}}'],
 ['#REDIRECT [[Constructed language]]\n\n{{Rcat shell|\n{{R from move}}\n{{R from misspelling|Artificial languages}}\n{{R from CamelCase}}\n{{Unprintworthy redirect}}\n}}'],
 ['#REDIRECT [[Principality of Aigues-Mortes]]\n\n{{R from alternative language|fr|en}}'],
 ['#REDIRECT [[Honoré Louit]]\n\n{{Redirect category shell|\n{{R from move}}\n}}'],
 ['#REDIRECT [[Elitism]]\n\n{{Redirect category shell|1=\n{{R from CamelCase}}\n}}'],
 ['#REDIRECT [[Abbreviation]]\n\n{{Redirect category shell|1=\n{{R from plural}}\n}}'],
 ['#REDIRECT [[Abecedarian]]\n\n{{Redirect category shell|1=


## Putting all together

So far, we've built a Faiss index using the misinformation abstract vectors we encoded with a sentence-DistilBERT model. That's helpful but in a real case scenario, we would have to work with unseen data. To query the index with an unseen query and retrieve its most relevant documents, we would have to do the following:

1. Encode the query with the same sentence-DistilBERT model we used for the rest of the abstract vectors.
2. Change its data type to float32.
3. Search the index with the encoded query.

Here, we will use the introduction of an article published on [HKS Misinformation Review](https://misinforeview.hks.harvard.edu/article/can-whatsapp-benefit-from-debunked-fact-checked-stories-to-reduce-misinformation/).


In [62]:
user_query = """
a state of disorder due to absence or non-recognition of authority or other controlling systems.
"""

In [63]:
# For convenience, I've wrapped all steps in the vector_search function.
# It takes four arguments: 
# A query, the sentence-level transformer, the Faiss index and the number of requested results
D, I = vector_search([user_query], model, index, num_results=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

L2 distance: [216.56982421875, 293.1074523925781, 294.1843566894531, 295.406005859375, 296.13623046875, 300.52825927734375, 303.629638671875, 308.8328857421875, 310.64935302734375, 310.9386901855469]

MAG paper IDs: [25, 69171675, 69171518, 69171648, 69171296, 69171335, 69171483, 69171363, 69171435, 69171299]


In [65]:
# Fetching the paper titles based on their index
id2details(final_df, I, 'title')

[['Autism'],
 ['Template:Taxonomy/Menkia'],
 ['Template:Taxonomy/Megalopelte'],
 ['Template:Taxonomy/Mengoana'],
 ['Draft:Anti-equilibrium theory'],
 ['Draft:William W. Stow'],
 ['Template:Taxonomy/Medora'],
 ['List of accidents and incidents involving the DC-3 in 1975'],
 ['How Was I to Know (song)'],
 ['List of accidents and incidents involving the DC-3 in the 1970s']]

In [ ]:
# Define project base directory
# Change the index from 1 to 0 if you run this on Google Colab
project_dir = Path('notebooks').resolve().parents[0]
print(project_dir)

# Serialise index and store it as a pickle
with open(f"{project_dir}/models/faiss_index.pickle", "wb") as h:
    pickle.dump(faiss.serialize_index(index), h)

/content/vector_engine
